In [12]:
import requests
from bs4 import BeautifulSoup

def fetchHTML(url):
    page = requests.get(url)
    html = BeautifulSoup(page.content, 'html.parser')
    return html

soup = fetchHTML('https://www.booking.com/searchresults.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGoAgO4Avqxi_wFwAIB0gIkYzMxM2IzM2MtNjRiOC00ZjA1LTg5ODEtOThmMDdkY2NhY2Iw2AIF4AIB&sid=be58553b8e68303c934b1ae3dd70e912&tmpl=searchresults&ac_click_type=b&ac_position=0&checkin_month=10&checkin_monthday=20&checkin_year=2020&checkout_month=10&checkout_monthday=31&checkout_year=2020&class_interval=1&dest_id=4127&dest_type=region&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&order=upsort_bh&postcard=0&raw_dest_type=region&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src_elem=sb&srpvid=6e7b50d26db201dc&ss=Goa%2C%20India&ss_all=0&ss_raw=goa&ssb=empty&sshis=0&top_ufis=1&rows=25')

In [13]:
hotelURL=[]
for anchor in soup.select('a.hotel_name_link'):
    if anchor.has_attr('href'):
        hotelURL.append('https://www.booking.com{}'.format(anchor['href'].translate({ord('\n'): None})))

In [ ]:
items=soup.find(id='results_prev_next').p.getText()
print(items)
import re
import math
item=int(re.sub("[^0-9]", "", items))
if item%25 and item>25==0:
    n=int((item/25))
if item>25 and item%25!=0:
    n=math.ceil(item/25)
else : n=0

In [ ]:
if n>0:
    for i in range(1,n):
        try:
            soup = fetchHTML('https://www.booking.com/searchresults.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaGyIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGoAgO4Avqxi_wFwAIB0gIkYzMxM2IzM2MtNjRiOC00ZjA1LTg5ODEtOThmMDdkY2NhY2Iw2AIF4AIB&sid=be58553b8e68303c934b1ae3dd70e912&tmpl=searchresults&ac_click_type=b&ac_position=0&checkin_month=10&checkin_monthday=20&checkin_year=2020&checkout_month=10&checkout_monthday=31&checkout_year=2020&class_interval=1&dest_id=4127&dest_type=region&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&order=upsort_bh&postcard=0&raw_dest_type=region&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src_elem=sb&srpvid=6e7b50d26db201dc&ss=Goa%2C%20India&ss_all=0&ss_raw=goa&ssb=empty&sshis=0&top_ufis=1&rows=25&offset={}'.format(i*25))
            for anchor in soup.select('a.hotel_name_link'):
                if anchor.has_attr('href'):
                    hotelURL.append('https://www.booking.com{}'.format(anchor['href'].translate({ord('\n'): None})))
        except:
            print(i)
    

In [14]:
import pandas as pd
pd.options.display.max_colwidth = 200
df=pd.DataFrame(data=hotelURL)
df.columns=['Hotel URL']

In [ ]:
name=[]
staffRating=[]
facilityRating=[]
cleanlinessRating=[]
comfortRating=[]
valueRating=[]
locationRating=[]
wifiRating=[]
overallRating=[]


for index,row in df.iterrows():
    
    try:
        
        soup = fetchHTML(row['Hotel URL'])
        name.append(soup.find(id='hp_hotel_name_reviews').getText().translate({ord('\n'): None}))

        try:
            rating = soup.find_all("li", class_="v2_review-scores__subscore")
            import re
            try:
                staffRating.append(re.sub('[^0-9,.]','', rating[0].getText()))
            except:
                staffRating.append(None)
            try:
                facilityRating.append(re.sub('[^0-9,.]','', rating[1].getText()))
            except:
                facilityRating.append(None)
            try:
                cleanlinessRating.append(re.sub('[^0-9,.]','', rating[2].getText()))
            except:
                cleanlinessRating.append(None)
            try:
                comfortRating.append(re.sub('[^0-9,.]','', rating[3].getText()))
            except:
                comfortRating.append(None)
            try:
                valueRating.append(re.sub('[^0-9,.]','', rating[4].getText()))
            except:
                valueRating.append(None)
            try:
                locationRating.append(re.sub('[^0-9,.]','', rating[5].getText()))
            except:
                locationRating.append(None)
            try:
                wifiRating.append(re.sub('[^0-9,.]','', rating[6].getText()))
            except:
                wifiRating.append(None)
            try:
                overallRating.append(soup.find_all('div',{'class':'bui-review-score__title'})[0].getText())
            except:
                overallRating.append(None)
                
        except:
            staffRating.append(None)
            facilityRating.append(None)
            cleanlinessRating.append(None)
            comfortRating.append(None)
            valueRating.append(None)
            locationRating.append(None)
            wifiRating.append(None)
            overallRating.append(None)
   
    except:
            df.drop(index, inplace=True)
            print('Couldnt fetch {}'.format(row['Hotel URL']))
            
df['Name']=name
df['Staff Rating']=staffRating
df['Facility Rating']=facilityRating
df['Cleanliness Rating']=cleanlinessRating
df['Comfort Rating']=comfortRating
df['Value for Money']=valueRating
df['Location Rating']=locationRating
df['Wifi Rating']=wifiRating
df['Overall Rating']=overallRating
df.head()

In [ ]:
import os
from PIL import Image

for row in df:
    soup = fetchHTML(row['Hotel URL'])
    imgSources = [img['src'] for img in soup.find_all("img", {"class" : "hide"})]
    
    path=os.path.join('./', '{}'.format(row['Name']))
    os.mkdir(path)
    i=0
    
    for src in imgSources:
        img = Image.open(requests.get(src, stream = True).raw)
        img.save('{}/{}.jpg'.format(path,str(i)), 'JPEG')
        i+=1